# The Audit Planning Agent - Draft Notebook

## Stage 1 - Ingestion
Ingesting trial balance and general ledger before mapping into a full fledge financial report

In [ ]:
# // import pathlib.Path, datetime, hashlib, pandas, yaml, pdfplumber